# Dialog with 3er almighty referee 

This notebook shows how the `DialogueAgent` and `DialogueSimulator` class make it easy to extend the [Two-Player Dungeons & Dragons example](https://python.langchain.com/en/latest/use_cases/agent_simulations/two_player_dnd.html) to multiple players.

The main difference between simulating two players and multiple players is in revising the schedule for when each agent speaks

To this end, we augment `DialogueSimulator` to take in a custom function that determines the schedule of which agent speaks. In the example below, each character speaks in round-robin fashion, with the storyteller interleaved between each player.

In [1]:
%load_ext autoreload
%autoreload 2
%autosave 60

Autosaving every 60 seconds


In [2]:
import os
#load from json .creds/PINECONE_API
import json
with open('.creds/PINECONE_API') as f:
    creds = json.load(f)
    PINECONE_API_KEY = creds['PINECONE_API_KEY']
    PINECONE_ENVIRONMENT = creds['PINECONE_ENVIRONMENT']
    OPENAI_API_KEY = creds['OPENAI_API_KEY']

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Sample Recruiter and Candidate Profiles

In [4]:
user_a = {
    'name': 'Recruiter', 
    'description': """id 1 title Data Analyst company London Approach description The ideal candidate for \
    the Data Analyst/Financial Analyst position will use their passion for data and analytics to provide \
    insights to the business covering a range of topics. They will be responsible for conducting both \
    recurring and ad hoc analysis for business users.Salary: $100-120kSchedule: 4 days a week onsite & 1 \
    day remote (usually Friday)ResponsibilitiesUnderstand the day-to-day issues that our business faces, \
    which can be better understood with dataCompile and analyze data related to business' issuesDevelop \
    clear visualizations to convey complicated data in a straightforward fashionCollaboratively build \
    financial models and reportsReview and track trendsInvestigate and research discrepanciesMaintain \
    system and code information in systemCompile and analyze dataPrepare and analyze operations and \
    financialsBuild trend reportsQualificationsBachelor's or Master's degree in Accounting or Finance2+ \
    years of relevant data analysis/financial analysis work onsite_remote onsite salary 100000 location_city \
    Los Angeles location_state CA Seniority level Associate Employment type Full-time Job function Finance \
    Industries Hospitals and Health Care posted_date 2022-11-21 link https://www.linkedin.com/jobs/view/data-\
    analyst-at-london-approach-3358861203?refId=7aKV74baVvH%2FRwNOmaw%2ByA%3D%3D&trackingId=9PyvYSdfQxwZH6r\
    UYZJw8g%3D%3D&position=9&pageNum=0&trk=public_jobs_jserp-result_search-card"""
}

user_b = {
    'name': 'Candidate', 
    'description': """ID: 1, Name: Saurav Joshi, URL: https://www.linkedin.com/in/saurav-joshi-ab58a9179/, \
    Title: M.S. Data Science @ USC'24 | GSoC'22 @ DBpedia | ICPC'20 Regionalist, Location: Los Angeles, CA, \
    Summary:  Connections 427 experience_count 2, Source: linkedin, Experience_count:  2, Connections: 427, \
    Title: Research Scientist, Location: Marina del Rey, California, Company: USC Information Sciences Institute, \
    Description: Developed a graph database by aggregating information from WikiData, PDF, and FoodKG, and \
    created a backend design for a knowledge-powered understanding application. Extracted meaningful insights \
    from PDFs and applied machine learning algorithms and statistical knowledge to build a recommendation system \
    that recommended food products that are more sustainable. Awarded Best Data Science Team Leader at USC\
    Information Sciences Institute CKIDS Datafest 2022., Title: Data Scientist, Location: Mumbai, Maharashtra, \
    Company: GSPL, Description: Developed an end-to-end sales prediction application for a commodity firm, \
    applied Information Gain, Chi-Square Test feature selection methods, and created an ARIMA model which \
    improved the prediction accuracy by 12%. Constructed"""
}

In [5]:
from src.agent_type import DialogueAgent
from src.simulation_type import DialogueSimulator

## Define roles and quest

In [6]:
character_names = [user_a['name'], user_b['name']]
storyteller_name = "God"
quest = "Find the defects and positive values of the other person for a potential job match. Propose career objectives and evaluate their compatibility. Determine the appealing and concerning aspects of this potential job match."
word_limit = 50  # word limit for task brainstorming

## Detail to date conditions

In [7]:
game_description = f"""Here is the topic for a Job Match Simulation: {quest}.
        The characters are: {*character_names,}.
        The conditions of the date is narrated by, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Job Match."
)


def generate_character_description(character_name):
    """TODO: Re think this to take the key value pairs from the database and lookup the description"""
    if character_name == user_a["name"]:
        character_description = user_a["description"]
    else:
        character_description = user_b["description"]
    return character_description


def generate_character_system_message(character_name, character_description):
    return SystemMessage(
        content=(
            f"""{game_description}
    Your name is {character_name}. 
    Your character description is as follows: {character_description}.
    You will propose actions you plan to take and {storyteller_name} will explain what happens when you take those actions.
    Speak in the first person from the perspective of {character_name}.
    For describing your own body movements, wrap your description in '*'.
    Do not change roles!
    Do not speak from the perspective of anyone else.
    Remember you are {character_name}.
    Stop speaking the moment you finish speaking from your perspective.
    Never forget to keep your response to {word_limit} words!
    Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_system_messages = [
    generate_character_system_message(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
         The objective is to have a god that acts to generate challenges or opportunities in the enviroment of a job match. Please reply with a creative description of the storyteller. {storyteller_name}, in {word_limit} words or less.
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
You are the storyteller, {storyteller_name}. 
Your description is as follows: {storyteller_description}.
The other players will propose actions to take and you will explain what happens when they take those actions.
Speak in the first person from the perspective of {storyteller_name}.
Do not change roles!
Do not speak from the perspective of anyone else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
"""
    )
)

In [8]:
print("Storyteller Description:")
print(storyteller_description+'\n')
for character_name, character_description in zip(
    character_names, character_descriptions
):
    print(f"{character_name} Description:")
    print(character_description)
    print()

Storyteller Description:
Dear God, as the storyteller of this job match simulation, we envision you as a wise and all-knowing guide, who can identify both the strengths and weaknesses of the Recruiter and Candidate. Your actions will shape the environment of this simulation, creating challenges and opportunities for them to test their compatibility and determine the appealing and concerning aspects of their potential job match.

Recruiter Description:
id 1 title Data Analyst company London Approach description The ideal candidate for     the Data Analyst/Financial Analyst position will use their passion for data and analytics to provide     insights to the business covering a range of topics. They will be responsible for conducting both     recurring and ad hoc analysis for business users.Salary: $100-120kSchedule: 4 days a week onsite & 1     day remote (usually Friday)ResponsibilitiesUnderstand the day-to-day issues that our business faces,     which can be better understood with dat

## Use an LLM to create an elaborate quest description

In [9]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the characters: {*character_names,}.
        Do not add anything else."""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Find the defects and positive values of the other person for a potential job match. Propose career objectives and evaluate their compatibility. Determine the appealing and concerning aspects of this potential job match.

Detailed quest:
Recruiter and Candidate, your task is to evaluate each other's personality, skills, and work experience, and identify whether the proposed career objectives align with your long-term goals in a fictional world where technology has advanced so far that employees are able to work from any location in the world. Determine the potential opportunities and challenges of this futuristic job match.



## Main Loop

In [10]:
characters = []
for character_name, character_system_message in zip(
    character_names, character_system_messages
):
    characters.append(
        DialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
        )
    )
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [11]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    """
    If the step is even, then select the storyteller
    Otherwise, select the other characters in a round-robin fashion.

    For example, with three characters with indices: 1 2 3
    The storyteller is index 0.
    Then the selected index will be as follows:

    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16

    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0
    """
    if step % 2 == 0:
        idx = 0
    else:
        idx = (step // 2) % (len(agents) - 1) + 1
    return idx

In [12]:
max_iters = 20
n = 0

simulator = DialogueSimulator(
    agents=[storyteller] + characters, selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(God): Recruiter and Candidate, your task is to evaluate each other's personality, skills, and work experience, and identify whether the proposed career objectives align with your long-term goals in a fictional world where technology has advanced so far that employees are able to work from any location in the world. Determine the potential opportunities and challenges of this futuristic job match.


(Recruiter): I would like to start by asking the Candidate about their experience with data analysis and financial modeling. Can you tell me about a project you worked on that required you to analyze complex data and present it in a clear and concise manner?


(God): As the Candidate answers, the Recruiter notices their attention to detail and ability to communicate complex information effectively. However, the Candidate also reveals a tendency to work independently, which may be a concern in a remote work environment.


(Candidate): Sure, I worked on a project where I developed an end-to-e